In [ ]:
## Notebook to prepare dataset

## Datasets ignorados

 1. brackets : Sem coluna daystofix
 2. camel : Sem coluna daystofix
 3. fabric8 :  Sem coluna daystofix
 4. jgroups :  Sem coluna daystofix
 5. neutron :  Sem coluna daystofix
 6. tomcat :  Sem coluna daystofix
 
**Investigar** como criar coluna daystofix 

In [ ]:
import os,gc
import numpy as np
from copy import copy
from glob import glob
import pandas as pd
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
BASE_PATH_DIR = '../data_raw'
SAVE_PATH_DIR = '../input/'

In [ ]:
dataset_dir = os.path.join(SAVE_PATH_DIR,"nova_preprocess.csv")

In [ ]:
from pandas import DataFrame


def move_columns(df: DataFrame, cols_to_move: list, new_index: int) -> DataFrame:
    """
    This method re-arranges the columns in a dataframe to place the desired columns at the desired index.
    ex Usage: df = move_columns(df, ['Rev'], 2)   
    :param df:
    :param cols_to_move: The names of the columns to move. They must be a list
    :param new_index: The 0-based location to place the columns.
    :return: Return a dataframe with the columns re-arranged
    """
    other = [c for c in df if c not in cols_to_move]
    start = other[0:new_index]
    end = other[new_index:]
    return df[start + cols_to_move + end]

In [ ]:
# Columns to read
columns = ['hashcode', 'fix', 'ns', 'nd', 'nf', 'entrophy', 'la', 'ld', 'lt',
                        'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp',
                        'containsbug', 'timestamp','daystofix']

In [ ]:
rename_dict = {
    'timestamp': 'arrival_time',
    'containsbug': 'class'}

In [ ]:
dtypes ={ 'fix': np.int8,
 'ns': np.int64,
 'nd': np.int64,
 'nf': np.int64,
 'entrophy': np.float64,
 'la': np.int64,
 'ld': np.int64,
 'lt': np.float64,
 'ndev': np.int64,
 'age': np.float64,
 'nuc': np.int64,
 'exp': np.float64,
 'rexp': np.float64,
 'sexp':np.int64,
 'class': np.int8,
 'arrival_time': np.int64,
 'daystofix': np.float64}

In [ ]:
# converting fix column to int
def f_bool(x):
    if x == 'False':
        x = np.int8(0)
    elif x == 'True':
        x = np.int8(1)
    return x

In [ ]:
dataset_list = []
for dirname, _, filenames in os.walk(BASE_PATH_DIR):
    for filename in filenames:
         
        if os.path.join(dirname, filename).split('.')[-1]=='csv':
               dataset_list.append(os.path.join(dirname, filename))

In [ ]:
for data in dataset_list:
    #print(data)
    df = pd.read_csv(data)
    df.columns = df.columns.str.strip()
    df['fix'] = df.fix.str.strip()
    df['fix'] = df.fix.apply(f_bool)
    df['containsbug'] = df.containsbug.str.strip()
    df['containsbug'] = df.containsbug.apply(f_bool)
    #removing hashcode column
    # df.drop(['hashcode'],axis=1,inplace=True)
    
    df.rename(columns=rename_dict, inplace=True)
    df = df.astype(dtypes)
    index_class = df.shape[1] -1
    df = move_columns(df,['class'],index_class)
    #save nova dataset preprocess
    file_name = data.split('\\')[-1].split('.')[-2]
    #print(file_name)
    if not os.path.isfile(os.path.join(SAVE_PATH_DIR,f'{file_name}'+'_preprocess.csv')):
        df.to_csv(os.path.join(SAVE_PATH_DIR,f'{file_name}'+'_preprocess.csv'),index=False)
        print('Dataset {} saved'.format(f'{file_name}'+'_preprocess.csv'))
        

In [ ]:
# TODO função ou class para formação do dataset com verification latency
df = pd.read_csv(dataset_dir)
df.shape